# Rock Muon Selection

In [46]:
import numpy as np

import h5py

import h5flow
from h5flow.data import dereference

from sklearn.cluster import DBSCAN

from sklearn.decomposition import PCA

import plotly.graph_objects as go

f_name = 'MiniRun3_1E19_RHC.flow_v6.00284.FLOW.h5'
f_manager = h5flow.data.H5FlowDataManager(f_name, 'r')

i_evt = 54

def cluster(i_evt):
    PromptHits_ev = f_manager["charge/events", "charge/calib_prompt_hits", i_evt]

    hits = np.zeros((PromptHits_ev.shape[1],5))

    for i in range(PromptHits_ev.shape[1]):
            hits[i][0] = PromptHits_ev['x'].data[0][i]
            hits[i][1] = PromptHits_ev['y'].data[0][i]
            hits[i][2] = PromptHits_ev['z'].data[0][i]

    hit_cluster = DBSCAN(eps = 80, min_samples = 1).fit(hits)

    for i in range(PromptHits_ev.shape[1]):
        hits[i][3] = hit_cluster.labels_[i]
        hits[i][4] = PromptHits_ev['id'].data[0][i] 
    return hits

hits = cluster(i_evt)

def PCAs(hits_of_track):
    pca = PCA(1) # 1 component

    pca.fit(hits_of_track)

    var = pca.explained_variance_ratio_

    return  var

def select_muon_track(hits):

    tracks = np.unique(hits[:,3]) #This gets how many tracks there are
    
    muon_hits = []
    
    a = []
    
    #Boundaries of the detector
    x_boundaries = np.array([-63.931, -3.069, 3.069, 63.931])
    y_boundaries = np.array([-19.8543, 103.8543])
    z_boundaries = np.array([-64.3163,  -2.6837, 2.6837, 64.3163])

    #Bottom face for each coordinate
    min_boundaries = np.array([x_boundaries[0], y_boundaries[0], z_boundaries[0]])

    #Top face for each coordinate
    max_boundaries = np.array([x_boundaries[-1], y_boundaries[-1], z_boundaries[-1]])

    for n in range(len(np.unique(tracks))):

        hits1 = hits[:,3] == n #get the hits with track number equal to n

        hits_with_track = hits[hits1] #position of the hits with their associated track number
        
        hitss_of_track = np.delete(hits_with_track, 4, axis = 1) #hits without their track number
        hits_of_track = np.delete(hitss_of_track, 3, axis = 1) #hits without their track number
        
        for i in range(len(max_boundaries)):
            #Does the track penetrate both z-boundaries?
            if min_boundaries[i]-2 < np.min(hits_of_track[:,i]) < (min_boundaries[i] + 2) and (max_boundaries[i] -2) < np.max(hits_of_track[:,i]) < max_boundaries[i]+2:
                a = PCAs(hits_of_track)
                if a > .9:
                    muon_hits.append(hits_with_track)
                    break
            elif min_boundaries[i]-2 < np.min(hits_of_track[:,i]) < (min_boundaries[i] + 2) and (max_boundaries[i-2] -2) < np.max(hits_of_track[:,i-2]) < max_boundaries[i-2]+2:
                a = PCAs(hits_of_track)
                if a > .9:
                    muon_hits.append(hits_with_track)
                    break
            elif min_boundaries[i]-2 < np.min(hits_of_track[:,i]) < (min_boundaries[i] + 2) and (max_boundaries[i-1] -2) < np.max(hits_of_track[:,i-1]) < max_boundaries[i-1]+2:
                a = PCAs(hits_of_track)
                if a > .9:
                    muon_hits.append(hits_with_track)
                    break
            elif min_boundaries[i]-2 < np.min(hits_of_track[:,i]) < (min_boundaries[i] + 2) and (min_boundaries[i-1] -2) < np.min(hits_of_track[:,i-1]) < min_boundaries[i-1]+2:
                a = PCAs(hits_of_track)
                if a > .9:
                    muon_hits.append(hits_with_track)
                    break
            elif min_boundaries[i]-2 < np.min(hits_of_track[:,i]) < (min_boundaries[i] + 2) and (min_boundaries[i-2] -2) < np.min(hits_of_track[:,i-2]) < min_boundaries[i-2]+2:
                a = PCAs(hits_of_track)
                if a > .9:
                    muon_hits.append(hits_with_track)    
                    break
    return muon_hits
muon_hits = select_muon_track(hits)
muon_hits                   
                                                                              
                                                                                                                              


[array([[-6.37234609e+01,  6.39482956e+01, -1.90895004e+01,
          0.00000000e+00,  4.59360000e+04],
        [-6.37074964e+01, -1.54203005e+01, -4.25896988e+01,
          0.00000000e+00,  4.59370000e+04],
        [-6.37074964e+01, -1.54203005e+01, -4.34765015e+01,
          0.00000000e+00,  4.59380000e+04],
        ...,
        [-5.18458545e+01, -1.45335007e+01,  1.15516996e+01,
          0.00000000e+00,  4.67060000e+04],
        [-5.18458545e+01, -1.49769001e+01,  1.19951000e+01,
          0.00000000e+00,  4.67070000e+04],
        [-5.17819964e+01, -1.45335007e+01,  1.19951000e+01,
          0.00000000e+00,  4.67160000e+04]]),
 array([[ 6.18715761e+01,  7.76937027e+01, -6.43162994e+01,
          1.00000000e+00,  4.61010000e+04],
        [ 6.18077180e+01,  7.76937027e+01, -6.38729019e+01,
          1.00000000e+00,  4.61060000e+04],
        [ 6.17917534e+01,  7.76937027e+01, -6.34294968e+01,
          1.00000000e+00,  4.61080000e+04],
        ...,
        [ 5.01695794e+01,  9.2325897

In [47]:
PromptHits_ev = f_manager["charge/events", "charge/calib_prompt_hits", i_evt]

Segs_PromptHits = f_manager["charge/raw_events","charge/calib_prompt_hits","charge/packets", "mc_truth/tracks", i_evt]

In [48]:
print(PromptHits_ev.data[0].shape)
print(len(np.unique(PromptHits_ev.data['z'])))
PromptHits_ev.data['z']

(848,)
280


array([[-19.08950043, -42.58969879, -43.47650146, -47.46709824,
        -19.53289986, -43.03310013, -19.97629929, -42.14630127,
        -41.70289993, -40.81610107, -41.2594986 , -20.86310005,
        -20.41970062, -40.37269974, -21.30649948, -39.48590088,
        -21.74989891, -38.15570068, -22.19330025, -37.71229935,
        -23.96689987, -39.9292984 , -23.52350044, -22.63669968,
        -38.59909821, -24.4102993 , -37.26889801, -36.82550049,
        -23.08009911, -24.85370064, -25.7404995 , -25.29710007,
        -35.49530029, -30.17449951, -26.18389893, -36.38210297,
        -35.93869781, -39.04249954, -35.05190277, -26.62730026,
        -27.95750046, -27.07069969, -28.40089989, -27.51409912,
        -28.84429932, -31.94809914, -31.06130028, -29.28770065,
        -30.61789894, -31.06130028, -32.39150238, -33.27829742,
        -31.94809914, -30.17449951, -31.50469971, -30.61789894,
        -34.60849762, -29.73110008, -32.39150238, -32.39150238,
        -32.8348999 , -30.17449951, -37.

In [49]:
print(Segs_PromptHits[0,:,0,0].data.shape)
print(len(np.unique(Segs_PromptHits[0,:,0,0].data['z'])))
Segs_PromptHits[0,:,0,0].data['z']

(848,)
62


array([-21.044655 , -45.261856 , -45.261856 , -45.261856 , -21.044655 ,
       -45.261856 , -21.044655 , -45.261856 , -41.057346 , -41.057346 ,
       -41.057346 , -21.044655 , -21.044655 , -41.057346 , -21.044655 ,
       -41.057346 , -21.044655 , -38.911987 , -21.044655 , -38.911987 ,
       -21.044655 , -41.057346 , -21.044655 , -21.044655 , -38.911987 ,
       -21.044655 , -34.51018  , -34.51018  , -21.044655 , -21.044655 ,
       -28.594795 , -28.594795 , -34.51018  , -34.51018  , -28.594795 ,
       -34.51018  , -34.51018  , -41.057346 , -34.51018  , -28.594795 ,
       -28.594795 , -28.594795 , -28.594795 , -28.594795 , -28.594795 ,
       -32.059006 , -32.059006 , -28.594795 , -28.594795 , -34.51018  ,
       -34.51018  , -34.51018  , -32.059006 , -34.51018  , -32.059006 ,
       -34.51018  , -34.51018  , -28.594795 , -34.89985  , -34.89985  ,
       -34.51018  , -28.594795 , -38.911987 , -28.775375 , -34.51018  ,
       -34.89985  , -32.543293 , -34.51018  , -34.51018  , -34.5

In [50]:
muon_hits[0][:,4] #hit ids

array([45936., 45937., 45938., 45939., 45940., 45941., 45942., 45943.,
       45944., 45945., 45946., 45947., 45948., 45949., 45950., 45951.,
       45952., 45953., 45954., 45955., 45956., 45957., 45958., 45959.,
       45960., 45961., 45962., 45963., 45964., 45965., 45966., 45967.,
       45968., 45969., 45970., 45971., 45972., 45973., 45974., 45975.,
       45976., 45977., 45978., 45979., 45980., 45981., 45982., 45983.,
       45984., 45985., 45986., 45987., 45988., 45989., 45990., 45991.,
       45992., 45993., 45994., 45995., 45996., 45997., 45998., 45999.,
       46000., 46001., 46002., 46003., 46004., 46005., 46006., 46007.,
       46008., 46009., 46010., 46011., 46012., 46013., 46014., 46015.,
       46016., 46017., 46018., 46019., 46020., 46021., 46022., 46023.,
       46024., 46025., 46026., 46027., 46028., 46029., 46030., 46031.,
       46032., 46033., 46034., 46035., 46036., 46037., 46038., 46039.,
       46040., 46041., 46042., 46043., 46044., 46045., 46046., 46047.,
      

In [51]:
a =PromptHits_ev['id'][0].data==45936

i = np.where(PromptHits_ev.data['id']==45936)[1][0]

print("Index of this hit id in PromptHits_ev, {}".format(i))

print("index {} for Segs_PromptHits".format(i), Segs_PromptHits[0,:,0,0][a].data)

print("index {} for PromptHits_ev".format(i), PromptHits_ev.data[0,0])

Index of this hit id in PromptHits_ev, 0
index 0 for Segs_PromptHits [(284042, 1000002842000141, 23637, -16.237602, 129679, 0.00552941, -25.851707, -64.04396, 64.11446, 586498, -13, -63.263374, 64.2452, 9.380656, 50400008.67292292, 50400008.6732448, 50400008.67308386, 9.646627, 0.00372793, 6, 12.855017, 2.0547445, 19.821356, 10.410264, 64.17983, -63.653667, -21.044655, 429517.78)]
index 0 for PromptHits_ev (45936, -63.72346093, 63.94829559, -19.08950043, 13., 504000088, 10.015625, 0.23636875)


In [52]:
x_boundaries = np.array([-63.931, -3.069, 3.069, 63.931])
y_boundaries = np.array([-19.8543, 103.8543])
z_boundaries = np.array([-64.3163,  -2.6837, 2.6837, 64.3163])

def draw_cathode_planes(x_boundaries, y_boundaries, z_boundaries, **kwargs):
    
    traces = []
    for i_z in range(int(len(z_boundaries)/2)):
        for i_x in range(int(len(x_boundaries)/2)):
            z, y = np.meshgrid(np.linspace(z_boundaries[i_z * 2], z_boundaries[i_z * 2 + 1], 2), np.linspace(y_boundaries.min(), y_boundaries.max(),2))
            x = (x_boundaries[i_x * 2] + x_boundaries[i_x * 2 + 1]) * 0.5 * np.ones(z.shape)

            traces.append(go.Surface(x=x, y=y, z=z, **kwargs))

    return traces

def draw_anode_planes(x_boundaries, y_boundaries, z_boundaries, **kwargs):
    
    traces = []
    for i_z in range(int(len(z_boundaries)/2)):
        for i_x in range(int(len(x_boundaries))):           
            z, y = np.meshgrid(np.linspace(z_boundaries[i_z * 2], z_boundaries[i_z * 2 + 1], 2), np.linspace(y_boundaries.min(), y_boundaries.max(),2))
            x = x_boundaries[i_x] * np.ones(z.shape)

            traces.append(go.Surface(x=x, y=y, z=z, **kwargs))
    
    return traces

In [53]:
fig = go.Figure()

##########################


##########################
# Draw the anodes
##########################
fig.add_traces(draw_anode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))

##########################
# Draw the prompt hits
##########################
PHits_traces = go.Scatter3d(
        x=Segs_PromptHits[0,:,0,0].data['x'].flatten(), y=Segs_PromptHits[0,:,0,0].data['y'].flatten(), z=Segs_PromptHits[0,:,0,0].data['z'].flatten(),
        marker_color=PromptHits_ev.data['E'].flatten()*1000,
        name='Segs_PromptHits',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=PromptHits_ev.data['E'].flatten()*1000,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>E:%{customdata:.3f}',
        )

fig.add_traces(PHits_traces)

fig.show()

fig1 = go.Figure()

# Draw the anodes
##########################
fig1.add_traces(draw_anode_planes(
    x_boundaries, y_boundaries, z_boundaries, 
    showscale=False,
    opacity=0.1,
    colorscale='ice',
))
##########################
# Draw the prompt hits
##########################
PHits_traces = go.Scatter3d(
        x=PromptHits_ev.data['x'].flatten(), y=PromptHits_ev.data['y'].flatten(), z=PromptHits_ev.data['z'].flatten(),
        marker_color=PromptHits_ev.data['E'].flatten()*1000,
        name='PromptHits_ev',
        mode='markers',
        visible='legendonly',
        marker_size=3,
        marker_symbol='square',
        showlegend=True,
        opacity=0.7,
        customdata=PromptHits_ev.data['E'].flatten()*1000,
        hovertemplate='<b>x:%{x:.3f}</b><br>y:%{y:.3f}<br>z:%{z:.3f}<br>E:%{customdata:.3f}',
        )
fig1.add_traces(PHits_traces)


fig1.show()